In [152]:
import numpy as np
from scipy.linalg import expm, sinm, cosm
from scipy import sparse
import scipy


import sys
sys.path.append('..')

In [153]:
L = 6
# Sz = np.zeros((3,3),dtype = np.complex128)
# Sx = np.zeros((3,3),dtype = np.complex128)
# Sy = np.zeros((3,3),dtype = np.complex128)

# Sz[0,0] = 1
# Sz[2,2] = -1
# Sx[0,1] = 1/np.sqrt(2)
# Sx[1,0] = 1/np.sqrt(2)
# Sx[1,2] = 1/np.sqrt(2)
# Sx[2,1] = 1/np.sqrt(2)
# Sy[0,1] = -1j/np.sqrt(2)
# Sy[1,0] = 1j/np.sqrt(2)
# Sy[1,2] = -1j/np.sqrt(2)
# Sy[2,1] = 1j/np.sqrt(2)

Sz = np.zeros([2,2])
Sz[0,0] = 1/2
Sz[1,1] = -1/2
Sx = np.zeros([2,2])
Sx[1,0] = 1/2
Sx[0,1] = 1/2
Sy = np.zeros([2,2], dtype=np.complex64)
Sy[1,0] = 1j/2
Sy[0,1] = -1j/2

# SzSz = sparse.kron(Sz,Sz,format='csr')
# SxSx = sparse.kron(Sx,Sx,format='csr')
# SySy = sparse.kron(Sy,Sy,format='csr')

# h = SzSz + SxSx + SySy

SzSz = np.kron(Sz,Sz).astype(np.float64)
SxSx = np.kron(Sx,Sx).astype(np.float64)
SySy = np.kron(Sy,Sy).astype(np.float64)

lh = SzSz + SxSx + SySy

/home/keisuke/miniconda3/envs/quspin/lib/python3.7/site-packages/ipykernel_launcher.py:35: ComplexWarning: Casting complex values to real discards the imaginary part


### exact diagonalization with original matrix

In [155]:
import functions as f
from importlib import reload
from scipy import sparse
import scipy.sparse.linalg
import scipy
reload(f)

H = sparse.csr_matrix((2**L, 2**L), dtype=np.float64)
for i in range(L):
    
    H += f.l2nl(lh, L, [i,(i+1)%L], sps = 2)
    H += f.l2nl(lh/2, L, [i,(i+2)%L], sps = 2)   

E, V = np.linalg.eigh(H.toarray())
print(E)

[-2.25000000e+00 -2.25000000e+00 -1.75000000e+00 -1.75000000e+00
 -1.75000000e+00 -1.56872930e+00 -1.56872930e+00 -1.56872930e+00
 -1.56872930e+00 -1.56872930e+00 -1.56872930e+00 -1.25000000e+00
 -1.25000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -2.50000000e-01
 -2.50000000e-01 -2.50000000e-01 -2.50000000e-01 -2.50000000e-01
 -2.50000000e-01 -3.51510989e-16 -2.90853943e-16  5.92721140e-17
  1.09018292e-16  1.61187079e-16  1.69567322e-16  2.27804931e-16
  4.27895823e-16  5.70719320e-16  7.03265248e-16  2.50000000e-01
  2.50000000e-01  2.50000000e-01  2.50000000e-01  2.50000000e-01
  2.50000000e-01  3.18729304e-01  3.18729304e-01  3.18729304e-01
  3.18729304e-01  3.18729304e-01  3.18729304e-01  1.00000000e+00
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00  2.25000000e+00  2.25000000e+00  2.25000000e+00
  2.25000000e+00  2.25000

/home/keisuke/miniconda3/envs/quspin/lib/python3.7/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


### rewritten with projection operators

In [22]:
LH = sparse.csr_matrix((2**3,2**3), dtype = np.float64)
i = 0
LH += f.l2nl(lh/2, 3, [i,(i+1)%L], sps = 2)
LH += f.l2nl(lh/2, 3, [(i+1)%L,(i+2)%L], sps = 2)
LH += f.l2nl(lh/2, 3, [i,(i+2)%L], sps = 2) 
LH = LH

In [171]:
LH_ = sparse.csr_matrix((2**3,2**3), dtype = np.float64)
i = 0
LH_ += f.l2nl(lh/2, 3, [(i+1)%L,i], sps = 2)
LH_ += f.l2nl(lh/2, 3, [(i+2)%L,(i+1)%L], sps = 2)
LH_ += f.l2nl(lh/2, 3, [(i+2)%L,i], sps = 2) 

In [23]:
E, V = np.linalg.eigh(LH.toarray())
print((E).round(6))

[-0.375 -0.375 -0.375 -0.375  0.375  0.375  0.375  0.375]


In [160]:
H = sparse.csr_matrix((2**L, 2**L), dtype=np.float64)
for i in range(L):
    
    H += f.l2nl(LH, L, [i,(i+1)%L,(i+2)%L], sps = 2) 
    
E, _ = scipy.sparse.linalg.eigsh(H, k = 10)
print(E)

[-2.25 -2.25  2.25 -1.75  2.25 -1.75  2.25  2.25  2.25  2.25]


### apply local unitary transformation

In [161]:
LH_prime = (LH + sparse.identity(2**3) * 3/8).toarray()
E, V = np.linalg.eigh(LH_prime)
V = V.real
print((E).round(6))

[-0.   -0.    0.    0.    0.75  0.75  0.75  0.75]


In [162]:
LH2 = V.T @ LH_prime @ V

In [167]:
U = np.kron(V, V)
C1 = sparse.csr_matrix((2**6, 2**6), dtype=np.float64)
C1 += f.l2nl(LH_prime, L, [1,2,3], sps = 2)
C1 += f.l2nl(LH_prime, L, [2,3,4], sps = 2)
C1 = C1.real.toarray()
C2 = U.T @ C1 @ U

In [168]:
U = np.kron(V, V)
C1_prime = sparse.csr_matrix((2**6, 2**6), dtype=np.float64)
C1_prime += f.l2nl(LH_prime, L, [4,5,0], sps = 2)
C1_prime += f.l2nl(LH_prime, L, [5,0,1], sps = 2)
C1_prime = C1_prime.real.toarray()
C2_prime = U.T @ C1_prime @ U

### symmetrization operator

In [182]:
p = np.zeros((2**3, 2**3))
for state in [[i,j,k] for i in range(2) for j in range(2) for k in range(2)]:
    n = f.state2num(np.array(state), sps = 2)
    m = f.state2num(np.array(state[::-1]), sps = 2)
    p[m,n] += 1/2
    p[n,n] += 1/2  
P = np.kron(p,p)

In [184]:
p

array([[1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.5, 0. , 0. , 0.5, 0. , 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0. , 0. , 0.5, 0. ],
       [0. , 0.5, 0. , 0. , 0.5, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0. , 0. , 0.5, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ]])

### apply symmetrization operator to $C1$ and $C1^{\prime}$ so that both become the same operator

In [ ]:
C1@P

In [120]:
index = []
for state in [[i,j] for i in range(4) for j in range(4)]:
    index.append(f.state2num(np.array(state), sps = 8))
index = np.array(index)

In [121]:
C3 = C2[:,index]

In [127]:
U3, E3, V3 = np.linalg.svd(C3)

In [147]:
V4 = np.zeros((4, 2**L))
V4[:, index] = V3[-4:]

In [148]:
C2 @ V4[0]

array([-6.93889390e-18,  1.11022302e-16,  9.71445147e-17,  4.51028104e-17,
       -1.47451495e-17, -5.26922256e-17,  0.00000000e+00,  2.08166817e-17,
       -6.93889390e-18,  5.89805982e-17, -9.36750677e-17,  4.85722573e-17,
       -3.29597460e-17, -3.12250226e-17,  1.73472348e-18,  1.56125113e-17,
       -3.64291930e-17,  7.63278329e-17,  1.24900090e-16,  3.12250226e-17,
        3.46944695e-18, -3.55618313e-17,  2.81491157e-19,  0.00000000e+00,
        3.12250226e-17, -1.45716772e-16, -8.50014503e-17, -3.12250226e-17,
       -7.02563008e-17,  1.38777878e-17, -4.33680869e-18, -3.46944695e-17,
       -2.42861287e-17,  1.12757026e-17,  3.81639165e-17, -2.60208521e-18,
        3.46944695e-18,  4.16333634e-17,  0.00000000e+00, -2.94902991e-17,
        4.16333634e-17, -1.73472348e-17, -3.46944695e-17, -3.46944695e-18,
        3.46944695e-18,  3.46944695e-18,  1.30104261e-17, -3.85185989e-33,
        0.00000000e+00,  6.93889390e-18,  2.16840434e-17,  4.33680869e-18,
        0.00000000e+00,  

In [150]:
V4.shape

(4, 64)